# Purpose
Create the dataframe need for the FSM. 
This dataframe will have a row for each bout and will include:
 - career average significant strike attempts per one minute (ca_ss_a_p1m)
 - combined significant strike attempts of both fighters per 1 minute (c_ss_a_p1m)
 

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from sqlalchemy import create_engine
from src import local

In [2]:
# Credentials
USER = local.user 
PASS = local.password
HOST = local.host
PORT = local.port

#create engine
engine = create_engine(f'postgresql://{USER}:{PASS}@{HOST}:{PORT}/match_finder')

# Get data from postgres database
### Join tables

- get the date from the events table
- use the bouts table to join the dates to the general table
- use the general table to join the bouts with the fighters


In [3]:
query = """
SELECT bout_id, fighter_id, round, ss_a, date
FROM rounds
JOIN bouts ON bouts.id = rounds.bout_id
JOIN events ON events.id = bouts.event_id
"""

data = pd.read_sql(query, engine)

In [4]:
data

,bout_id,fighter_id,round,ss_a,date
0,11f715fa5e825e51,e1147d3d2dabe1ce,1,30,"July 25, 2020"
1,11f715fa5e825e51,e1147d3d2dabe1ce,2,30,"July 25, 2020"
2,11f715fa5e825e51,e1147d3d2dabe1ce,3,32,"July 25, 2020"
3,11f715fa5e825e51,e1147d3d2dabe1ce,4,34,"July 25, 2020"
4,11f715fa5e825e51,e1147d3d2dabe1ce,5,31,"July 25, 2020"
...,...,...,...,...,...
26209,cecdc0da584274b9,a5c53b3ddb31cc7d,1,0,"November 12, 1993"
26210,2d2bbc86e941e05c,598a58db87b890ee,1,27,"November 12, 1993"
26211,2d2bbc86e941e05c,d3711d3784b76255,1,28,"November 12, 1993"
26212,567a09fd200cfa05,279093302a6f44b3,1,5,"November 12, 1993"


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26214 entries, 0 to 26213
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   bout_id     26214 non-null  object
 1   fighter_id  26214 non-null  object
 2   round       26214 non-null  object
 3   ss_a        26214 non-null  int64 
 4   date        26214 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.0+ MB


## Convert Date to datetime

In [6]:
data['date'] = pd.to_datetime(data['date'])

In [7]:
data

,bout_id,fighter_id,round,ss_a,date
0,11f715fa5e825e51,e1147d3d2dabe1ce,1,30,2020-07-25
1,11f715fa5e825e51,e1147d3d2dabe1ce,2,30,2020-07-25
2,11f715fa5e825e51,e1147d3d2dabe1ce,3,32,2020-07-25
3,11f715fa5e825e51,e1147d3d2dabe1ce,4,34,2020-07-25
4,11f715fa5e825e51,e1147d3d2dabe1ce,5,31,2020-07-25
...,...,...,...,...,...
26209,cecdc0da584274b9,a5c53b3ddb31cc7d,1,0,1993-11-12
26210,2d2bbc86e941e05c,598a58db87b890ee,1,27,1993-11-12
26211,2d2bbc86e941e05c,d3711d3784b76255,1,28,1993-11-12
26212,567a09fd200cfa05,279093302a6f44b3,1,5,1993-11-12


## Create fighter-bout instance dataframe

A fighter-bout instance represents one fighter in one bout.
 - The same fighter has exactly one fighter-bout instance for every single bout he has been in. 
 - Every bout has exactly two fighter-bout instances, one for each fighter in the bout. 
  
In this case a fighter-bout instance is assigned a unique identifier comprised of the bout_id combined with the fighter_link.

##### Create a unique identifier for each fighter in each bout and create a groupby object

In [17]:
data['fighter_bout_inst'] = data['bout_id'] + data['fighter_link']
fighter_bout_inst_group = data.groupby(['fighter_bout_inst'])

##### 

In [18]:
TSSL = fighter_bout_inst_group.TSSL.sum()

In [19]:
date = fighter_bout_inst_group['Date'].max()

In [20]:
fighter_link = fighter_bout_inst_group['fighter_link'].max()

In [21]:
bout_id = fighter_bout_inst_group['bout_id'].max()

In [22]:
fighter_bout_inst = pd.DataFrame(dict(bout_id = bout_id, fighter_link = fighter_link, date = date, TSSL = TSSL))
fighter_bout_inst

,bout_id,fighter_link,date,TSSL
fighter_bout_inst,,,,
000da3152b7b5ab16da99156486ed6c2,000da3152b7b5ab1,6da99156486ed6c2,2006-07-08,35
000da3152b7b5ab1d1a1314976c50bef,000da3152b7b5ab1,d1a1314976c50bef,2006-07-08,19
0019ec81fd706ade326f94d6cfb1bf25,0019ec81fd706ade,326f94d6cfb1bf25,2019-10-18,26
0019ec81fd706ade85073dbd1be65ed9,0019ec81fd706ade,85073dbd1be65ed9,2019-10-18,54
0027e179b743c86c3aa794cbe1e3484b,0027e179b743c86c,3aa794cbe1e3484b,2015-03-14,9
...,...,...,...,...
ffe629a5232a878bb361180739bed4b0,ffe629a5232a878b,b361180739bed4b0,2003-06-06,0
ffea776913451b6d22a92d7f62195791,ffea776913451b6d,22a92d7f62195791,2015-02-28,11
ffea776913451b6d75e5fec9f72910ef,ffea776913451b6d,75e5fec9f72910ef,2015-02-28,2


## Calculate ASSL

For this I will need a function that takes in a fighter_link and a date and calculates the ASSL up until that date.

In [23]:
def calculate_ASSL(fighter_link, date, df):
    """
    input: fighter_link - str, a unique fighter identifier
           date - datetime64, cut off date, ASSL will be calculated using every fight up until this date
           df - dataframe, a fighter-instance table containing the TSSL
    output: float, the ASSL for the fighter up until the date
    """
    fighter_history = df[(df['fighter_link']==fighter_link)&
                         (df['date']<date)]
    fighter_ASSL = fighter_history.TSSL.mean()
    return fighter_ASSL

In [24]:
ASSL = fighter_bout_inst.apply(lambda row: calculate_ASSL(row['fighter_link'], row['date'], fighter_bout_inst), axis=1)

In [25]:
ASSL

fighter_bout_inst
000da3152b7b5ab16da99156486ed6c2    10.333333
000da3152b7b5ab1d1a1314976c50bef    16.333333
0019ec81fd706ade326f94d6cfb1bf25          NaN
0019ec81fd706ade85073dbd1be65ed9    26.333333
0027e179b743c86c3aa794cbe1e3484b    20.000000
                                      ...    
ffe629a5232a878bb361180739bed4b0     9.062500
ffea776913451b6d22a92d7f62195791    30.500000
ffea776913451b6d75e5fec9f72910ef    23.583333
fffa21388cdd78b75d7bdab5e03e3216    17.272727
fffa21388cdd78b7c80095f6092271a7    31.333333
Length: 11362, dtype: float64

In [26]:
fighter_bout_inst['ASSL'] = ASSL
fighter_bout_inst

,bout_id,fighter_link,date,TSSL,ASSL
fighter_bout_inst,,,,,
000da3152b7b5ab16da99156486ed6c2,000da3152b7b5ab1,6da99156486ed6c2,2006-07-08,35,10.333333
000da3152b7b5ab1d1a1314976c50bef,000da3152b7b5ab1,d1a1314976c50bef,2006-07-08,19,16.333333
0019ec81fd706ade326f94d6cfb1bf25,0019ec81fd706ade,326f94d6cfb1bf25,2019-10-18,26,NaN
0019ec81fd706ade85073dbd1be65ed9,0019ec81fd706ade,85073dbd1be65ed9,2019-10-18,54,26.333333
0027e179b743c86c3aa794cbe1e3484b,0027e179b743c86c,3aa794cbe1e3484b,2015-03-14,9,20.000000
...,...,...,...,...,...
ffe629a5232a878bb361180739bed4b0,ffe629a5232a878b,b361180739bed4b0,2003-06-06,0,9.062500
ffea776913451b6d22a92d7f62195791,ffea776913451b6d,22a92d7f62195791,2015-02-28,11,30.500000
ffea776913451b6d75e5fec9f72910ef,ffea776913451b6d,75e5fec9f72910ef,2015-02-28,2,23.583333


I want to see why I got that NaN in the third row

In [27]:
data[data['fighter_bout_inst'] == '0019ec81fd706ade326f94d6cfb1bf25']

,bout_id,fighter_link,sig_str,Date,round,TSSL,fighter_bout_inst
1841,0019ec81fd706ade,326f94d6cfb1bf25,9 of 32,2019-10-18,1,9,0019ec81fd706ade326f94d6cfb1bf25
1842,0019ec81fd706ade,326f94d6cfb1bf25,7 of 35,2019-10-18,2,7,0019ec81fd706ade326f94d6cfb1bf25
1843,0019ec81fd706ade,326f94d6cfb1bf25,10 of 30,2019-10-18,3,10,0019ec81fd706ade326f94d6cfb1bf25


With this method of calculating the ASSL, we won't get values for fighters who have not fought for the UFC before. I need to either remove them or find another way to fill them in.

In [28]:
fighter_bout_inst.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11362 entries, 000da3152b7b5ab16da99156486ed6c2 to fffa21388cdd78b7c80095f6092271a7
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   bout_id       11362 non-null  object        
 1   fighter_link  11362 non-null  object        
 2   date          11362 non-null  datetime64[ns]
 3   TSSL          11362 non-null  int64         
 4   ASSL          9267 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 852.6+ KB


In [29]:
len(fighter_bout_inst[fighter_bout_inst.ASSL.isna()])/len(fighter_bout_inst)

0.18438655166343954

About 18% of my data is made up of first time fighters. For now, we'll remove them. If we just drop the NA's then we risk leaving their opponent in the dataframe, which would be useless for our purposes. This means that up to 36% of our fights have a fighter who has not fought for the UFC before.

In [30]:
debut_bouts = list(fighter_bout_inst[fighter_bout_inst.ASSL.isna()].bout_id.unique())

print('debut bouts:',len(debut_bouts))
print(len(debut_bouts)/len(fighter_bout_inst.bout_id.unique()))

debut bouts: 1643
0.2892096461890512


Almost 30% of our data includes debut fights.

In [31]:
def black_list_entry(entry, black_list):
    return entry not in black_list

In [33]:
mask = fighter_bout_inst['bout_id'].map(lambda x: black_list_entry(x, debut_bouts))

In [34]:
fighter_bout_inst = fighter_bout_inst[mask]

## Create the final dataframe

First I will get a list af all bout ids. Then I will create a dataframe with the first row and another dataframe with the second row. Then I will join those dataframes along the column axis.

In [35]:
fighter_bout_inst['inst_id'] = fighter_bout_inst.index

fighter_bout_inst

<ipython-input-35-c6a2a9fc723d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fighter_bout_inst['inst_id'] = fighter_bout_inst.index


,bout_id,fighter_link,date,TSSL,ASSL,inst_id
fighter_bout_inst,,,,,,
000da3152b7b5ab16da99156486ed6c2,000da3152b7b5ab1,6da99156486ed6c2,2006-07-08,35,10.333333,000da3152b7b5ab16da99156486ed6c2
000da3152b7b5ab1d1a1314976c50bef,000da3152b7b5ab1,d1a1314976c50bef,2006-07-08,19,16.333333,000da3152b7b5ab1d1a1314976c50bef
0027e179b743c86c3aa794cbe1e3484b,0027e179b743c86c,3aa794cbe1e3484b,2015-03-14,9,20.000000,0027e179b743c86c3aa794cbe1e3484b
0027e179b743c86c91ea901c458e95dd,0027e179b743c86c,91ea901c458e95dd,2015-03-14,22,32.250000,0027e179b743c86c91ea901c458e95dd
002921976d27b7dab4ad3a06ee4d660c,002921976d27b7da,b4ad3a06ee4d660c,2014-12-13,17,38.600000,002921976d27b7dab4ad3a06ee4d660c
...,...,...,...,...,...,...
ffe629a5232a878bb361180739bed4b0,ffe629a5232a878b,b361180739bed4b0,2003-06-06,0,9.062500,ffe629a5232a878bb361180739bed4b0
ffea776913451b6d22a92d7f62195791,ffea776913451b6d,22a92d7f62195791,2015-02-28,11,30.500000,ffea776913451b6d22a92d7f62195791
ffea776913451b6d75e5fec9f72910ef,ffea776913451b6d,75e5fec9f72910ef,2015-02-28,2,23.583333,ffea776913451b6d75e5fec9f72910ef


In [36]:
fighter_0 = list(fighter_bout_inst.groupby('bout_id').inst_id.max())

In [37]:
fighter_1 = list(fighter_bout_inst.groupby('bout_id').inst_id.min())

In [38]:
mask = fighter_bout_inst['inst_id'].map(lambda x: black_list_entry(x, fighter_0))
fighter_bout_inst_0 = fighter_bout_inst[mask]
fighter_bout_inst_0

,bout_id,fighter_link,date,TSSL,ASSL,inst_id
fighter_bout_inst,,,,,,
000da3152b7b5ab16da99156486ed6c2,000da3152b7b5ab1,6da99156486ed6c2,2006-07-08,35,10.333333,000da3152b7b5ab16da99156486ed6c2
0027e179b743c86c3aa794cbe1e3484b,0027e179b743c86c,3aa794cbe1e3484b,2015-03-14,9,20.000000,0027e179b743c86c3aa794cbe1e3484b
002921976d27b7dab4ad3a06ee4d660c,002921976d27b7da,b4ad3a06ee4d660c,2014-12-13,17,38.600000,002921976d27b7dab4ad3a06ee4d660c
002c1562708ac30722a92d7f62195791,002c1562708ac307,22a92d7f62195791,2014-05-24,38,32.600000,002c1562708ac30722a92d7f62195791
002cb1bb411c5f6022e47b53e4ceb27c,002cb1bb411c5f60,22e47b53e4ceb27c,2006-03-04,21,20.200000,002cb1bb411c5f6022e47b53e4ceb27c
...,...,...,...,...,...,...
ffbc12e4f821ec683591d0d5d382a381,ffbc12e4f821ec68,3591d0d5d382a381,2014-02-15,19,26.333333,ffbc12e4f821ec683591d0d5d382a381
ffd3e3d37cba32da7413b80dbb0f8f9f,ffd3e3d37cba32da,7413b80dbb0f8f9f,2014-10-25,26,28.500000,ffd3e3d37cba32da7413b80dbb0f8f9f
ffe629a5232a878b08ae5cd9aef7ddd3,ffe629a5232a878b,08ae5cd9aef7ddd3,2003-06-06,1,8.000000,ffe629a5232a878b08ae5cd9aef7ddd3


In [39]:
mask = fighter_bout_inst['inst_id'].map(lambda x: black_list_entry(x, fighter_1))
fighter_bout_inst_1 = fighter_bout_inst[mask]
fighter_bout_inst_1

,bout_id,fighter_link,date,TSSL,ASSL,inst_id
fighter_bout_inst,,,,,,
000da3152b7b5ab1d1a1314976c50bef,000da3152b7b5ab1,d1a1314976c50bef,2006-07-08,19,16.333333,000da3152b7b5ab1d1a1314976c50bef
0027e179b743c86c91ea901c458e95dd,0027e179b743c86c,91ea901c458e95dd,2015-03-14,22,32.250000,0027e179b743c86c91ea901c458e95dd
002921976d27b7daebc1f40e00e0c481,002921976d27b7da,ebc1f40e00e0c481,2014-12-13,2,20.230769,002921976d27b7daebc1f40e00e0c481
002c1562708ac30744470bfd9483c7ad,002c1562708ac307,44470bfd9483c7ad,2014-05-24,22,23.000000,002c1562708ac30744470bfd9483c7ad
002cb1bb411c5f60d897897060f10a3a,002cb1bb411c5f60,d897897060f10a3a,2006-03-04,127,37.833333,002cb1bb411c5f60d897897060f10a3a
...,...,...,...,...,...,...
ffbc12e4f821ec687a703c565ccaa18f,ffbc12e4f821ec68,7a703c565ccaa18f,2014-02-15,16,18.000000,ffbc12e4f821ec687a703c565ccaa18f
ffd3e3d37cba32da92a9aa9c93192871,ffd3e3d37cba32da,92a9aa9c93192871,2014-10-25,47,27.000000,ffd3e3d37cba32da92a9aa9c93192871
ffe629a5232a878bb361180739bed4b0,ffe629a5232a878b,b361180739bed4b0,2003-06-06,0,9.062500,ffe629a5232a878bb361180739bed4b0


In [40]:
fsm_dataframe = pd.merge(fighter_bout_inst_0, fighter_bout_inst_1, on='bout_id', suffixes=('_0', '_1'))
fsm_dataframe

,bout_id,fighter_link_0,date_0,TSSL_0,ASSL_0,inst_id_0,fighter_link_1,date_1,TSSL_1,ASSL_1,inst_id_1
0,000da3152b7b5ab1,6da99156486ed6c2,2006-07-08,35,10.333333,000da3152b7b5ab16da99156486ed6c2,d1a1314976c50bef,2006-07-08,19,16.333333,000da3152b7b5ab1d1a1314976c50bef
1,0027e179b743c86c,3aa794cbe1e3484b,2015-03-14,9,20.000000,0027e179b743c86c3aa794cbe1e3484b,91ea901c458e95dd,2015-03-14,22,32.250000,0027e179b743c86c91ea901c458e95dd
2,002921976d27b7da,b4ad3a06ee4d660c,2014-12-13,17,38.600000,002921976d27b7dab4ad3a06ee4d660c,ebc1f40e00e0c481,2014-12-13,2,20.230769,002921976d27b7daebc1f40e00e0c481
3,002c1562708ac307,22a92d7f62195791,2014-05-24,38,32.600000,002c1562708ac30722a92d7f62195791,44470bfd9483c7ad,2014-05-24,22,23.000000,002c1562708ac30744470bfd9483c7ad
4,002cb1bb411c5f60,22e47b53e4ceb27c,2006-03-04,21,20.200000,002cb1bb411c5f6022e47b53e4ceb27c,d897897060f10a3a,2006-03-04,127,37.833333,002cb1bb411c5f60d897897060f10a3a
...,...,...,...,...,...,...,...,...,...,...,...
4033,ffbc12e4f821ec68,3591d0d5d382a381,2014-02-15,19,26.333333,ffbc12e4f821ec683591d0d5d382a381,7a703c565ccaa18f,2014-02-15,16,18.000000,ffbc12e4f821ec687a703c565ccaa18f
4034,ffd3e3d37cba32da,7413b80dbb0f8f9f,2014-10-25,26,28.500000,ffd3e3d37cba32da7413b80dbb0f8f9f,92a9aa9c93192871,2014-10-25,47,27.000000,ffd3e3d37cba32da92a9aa9c93192871
4035,ffe629a5232a878b,08ae5cd9aef7ddd3,2003-06-06,1,8.000000,ffe629a5232a878b08ae5cd9aef7ddd3,b361180739bed4b0,2003-06-06,0,9.062500,ffe629a5232a878bb361180739bed4b0
4036,ffea776913451b6d,22a92d7f62195791,2015-02-28,11,30.500000,ffea776913451b6d22a92d7f62195791,75e5fec9f72910ef,2015-02-28,2,23.583333,ffea776913451b6d75e5fec9f72910ef


In [41]:
fsm_dataframe['TSSL_bout'] = fsm_dataframe['TSSL_0'] + fsm_dataframe['TSSL_1']
fsm_dataframe

,bout_id,fighter_link_0,date_0,TSSL_0,ASSL_0,inst_id_0,fighter_link_1,date_1,TSSL_1,ASSL_1,inst_id_1,TSSL_bout
0,000da3152b7b5ab1,6da99156486ed6c2,2006-07-08,35,10.333333,000da3152b7b5ab16da99156486ed6c2,d1a1314976c50bef,2006-07-08,19,16.333333,000da3152b7b5ab1d1a1314976c50bef,54
1,0027e179b743c86c,3aa794cbe1e3484b,2015-03-14,9,20.000000,0027e179b743c86c3aa794cbe1e3484b,91ea901c458e95dd,2015-03-14,22,32.250000,0027e179b743c86c91ea901c458e95dd,31
2,002921976d27b7da,b4ad3a06ee4d660c,2014-12-13,17,38.600000,002921976d27b7dab4ad3a06ee4d660c,ebc1f40e00e0c481,2014-12-13,2,20.230769,002921976d27b7daebc1f40e00e0c481,19
3,002c1562708ac307,22a92d7f62195791,2014-05-24,38,32.600000,002c1562708ac30722a92d7f62195791,44470bfd9483c7ad,2014-05-24,22,23.000000,002c1562708ac30744470bfd9483c7ad,60
4,002cb1bb411c5f60,22e47b53e4ceb27c,2006-03-04,21,20.200000,002cb1bb411c5f6022e47b53e4ceb27c,d897897060f10a3a,2006-03-04,127,37.833333,002cb1bb411c5f60d897897060f10a3a,148
...,...,...,...,...,...,...,...,...,...,...,...,...
4033,ffbc12e4f821ec68,3591d0d5d382a381,2014-02-15,19,26.333333,ffbc12e4f821ec683591d0d5d382a381,7a703c565ccaa18f,2014-02-15,16,18.000000,ffbc12e4f821ec687a703c565ccaa18f,35
4034,ffd3e3d37cba32da,7413b80dbb0f8f9f,2014-10-25,26,28.500000,ffd3e3d37cba32da7413b80dbb0f8f9f,92a9aa9c93192871,2014-10-25,47,27.000000,ffd3e3d37cba32da92a9aa9c93192871,73
4035,ffe629a5232a878b,08ae5cd9aef7ddd3,2003-06-06,1,8.000000,ffe629a5232a878b08ae5cd9aef7ddd3,b361180739bed4b0,2003-06-06,0,9.062500,ffe629a5232a878bb361180739bed4b0,1
4036,ffea776913451b6d,22a92d7f62195791,2015-02-28,11,30.500000,ffea776913451b6d22a92d7f62195791,75e5fec9f72910ef,2015-02-28,2,23.583333,ffea776913451b6d75e5fec9f72910ef,13


In [42]:
fsm_dataframe = fsm_dataframe.loc[:,['ASSL_0', 'ASSL_1', 'TSSL_bout']]

In [43]:
fsm_dataframe

,ASSL_0,ASSL_1,TSSL_bout
0,10.333333,16.333333,54
1,20.000000,32.250000,31
2,38.600000,20.230769,19
3,32.600000,23.000000,60
4,20.200000,37.833333,148
...,...,...,...
4033,26.333333,18.000000,35
4034,28.500000,27.000000,73
4035,8.000000,9.062500,1
4036,30.500000,23.583333,13


In [44]:
fsm_dataframe.to_csv('../../data/fsm_data.csv')